# Dataset Benchmarking

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import qcodes as qc
from qcodes import ParamSpec, new_data_set, new_experiment
import numpy as np

/Users/jhn/.pyenv/versions/3.6.4/envs/qcodes-3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
qc.config.core.db_location

'./experiments.db'

## Setup

In [3]:
exp = new_experiment("benchmarking", sample_name="the sample is a lie")
exp

benchmarking#the sample is a lie#11@./experiments.db
----------------------------------------------------

Now we can create a dataset. Note two things:

    - if we don't specfiy a exp_id, but we have an experiment in the experiment container the dataset will go into that one.
    - dataset can be created from the experiment object
    

In [4]:
dataSet = new_data_set("benchmark_data")
exp

benchmarking#the sample is a lie#11@./experiments.db
----------------------------------------------------
120-benchmark_data-1-None-0

In this benchmark we will assueme that we are doing a 2D loop and investigate the performance implications of writing to the dataset

In [5]:
x_shape = 100
y_shape = 100

## Baseline: Generate data

In [7]:
%%time
for x in range(x_shape):
    for y in range(y_shape):
        z = np.random.random_sample(1)

CPU times: user 30.2 ms, sys: 6.67 ms, total: 36.9 ms
Wall time: 36.3 ms


and store in memory

In [8]:
x_data = np.zeros((x_shape, y_shape))
y_data = np.zeros((x_shape, y_shape))
z_data = np.zeros((x_shape, y_shape))

In [9]:
%%time
for x in range(x_shape):
    for y in range(y_shape):
        x_data[x,y] = x
        y_data[x,y] = y
        z_data[x,y] = np.random.random_sample()

CPU times: user 16.1 ms, sys: 2.91 ms, total: 19 ms
Wall time: 17.1 ms


## Add to dataset inside double loop

In [10]:
double_dataset = new_data_set("doubledata", specs=[ParamSpec("x", "numeric"),
                                                   ParamSpec("y", "numeric"),
                                                   ParamSpec('z', "numeric")])

Note that this is so slow that we are only doing a 10th of the computation

In [18]:
%%time
for x in range(x_shape//10):
    for y in range(y_shape):
        double_dataset.add_result({"x": x, 'y': y, 'z': np.random.random_sample()})

CPU times: user 1.61 s, sys: 1.29 s, total: 2.9 s
Wall time: 3.36 s


## Add the data in outer loop and store as np array

In [12]:
single_dataset = new_data_set("singledata", specs=[ParamSpec("x", "array"),
                                                   ParamSpec("y", "array"),
                                                   ParamSpec('z', "array")])
x_data = np.zeros((y_shape))
y_data = np.zeros((y_shape))
z_data = np.zeros((y_shape))

In [13]:
%%time
for x in range(x_shape):
    for y in range(y_shape):
        x_data[y] = x
        y_data[y] = y
        z_data[y] = np.random.random_sample(1)
    single_dataset.add_result({"x": x_data, 'y': y_data, 'z': z_data})

CPU times: user 257 ms, sys: 147 ms, total: 404 ms
Wall time: 463 ms


## Save once after loop

In [14]:
zero_dataset = new_data_set("zerodata", specs=[ParamSpec("x", "array"),
                                                   ParamSpec("y", "array"),
                                                   ParamSpec('z', "array")])
x_data = np.zeros((x_shape, y_shape))
y_data = np.zeros((x_shape, y_shape))
z_data = np.zeros((x_shape, y_shape))

In [16]:
%%time
for x in range(x_shape):
    for y in range(y_shape):
        x_data[x,y] = x
        y_data[x,y] = y
        z_data[x,y] = np.random.random_sample(1)
zero_dataset.add_result({'x':x_data, 'y':y_data, 'z':z_data})

CPU times: user 39.3 ms, sys: 9.85 ms, total: 49.2 ms
Wall time: 44.9 ms


## Array parameter

In [15]:
array1D_dataset = new_data_set("array1Ddata", specs=[ParamSpec("x", "array"),
                                                     ParamSpec("y", "array"),
                                                     ParamSpec('z', "array")])
y_setpoints = np.arange(y_shape)

In [16]:
%%timeit
for x in range(x_shape):
    x_data[x,:] = x
    array1D_dataset.add_result({'x':x_data[x,:], 'y':y_setpoints, 'z':np.random.random_sample(y_shape)})

215 ms ± 2.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
x_data = np.zeros((x_shape, y_shape))
y_data = np.zeros((x_shape, y_shape))
z_data = np.zeros((x_shape, y_shape))
y_setpoints = np.arange(y_shape)

In [18]:
array0D_dataset = new_data_set("array0Ddata", specs=[ParamSpec("x", "array"),
                                                     ParamSpec("y", "array"),
                                                     ParamSpec('z', "array")])

In [19]:
%%timeit
for x in range(x_shape):
    x_data[x,:] = x
    y_data[x,:] = y_setpoints
    z_data[x,:] = np.random.random_sample(y_shape)
array0D_dataset.add_result({'x':x_data, 'y':y_data, 'z':z_data})

3.48 ms ± 76.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Insert many

In [20]:
data = []
for i in range(100):
    for j in range(100):
        data.append({'x': i, 'y':j, 'z':np.random.random_sample()})

In [21]:
many_Data = new_data_set("many_data", specs=[ParamSpec("x", "numeric"),
                                             ParamSpec("y", "numeric"),
                                             ParamSpec("z", "numeric")])

In [22]:
%%timeit
many_Data.add_results(data)

10.7 ms ± 338 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
